# Designing Typeahead Suggestion

Typeahead is a real-time suggestion service which recommends terms to users as they enter text for searching.

As the user types into the search box, it tries to predict the query based on the characters the users has entered, and gives a list of suggestions to complete the query.

It's not about speeding up the users' search but to help the user articulate their search queries better.


## 1. Requirements and Goals of the System
**Functional requirements:** As the user types in their search query, our service should suggest top 10 terms starting with whatever the user typed.

**Non-functional requirements:** The suggestions should appear in real-time, allowing the user to see it in about 200ms.
